# Preprocessing, GridSearch and Training CNN-3, CNN-6 and CNN-8 on Open Data

In [1]:
import numpy as np
from numpy import array
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import math

import matplotlib.pylab as plt
from matplotlib.pyplot import cm
import seaborn as sns
import itertools
import collections
from collections import OrderedDict, Counter

import scipy.stats as stats
from scipy.stats import normaltest, kstest, ttest_ind, ttest_rel, mannwhitneyu, wilcoxon, levene, bartlett
from scipy.stats import chi2_contingency, fisher_exact, mode, pearsonr, f_oneway, kruskal, spearmanr

from rfpimp import *

from datetime import datetime
import re
from seaborn import heatmap
import random
import statsmodels.distributions.empirical_distribution as edf
from scipy.interpolate import interp1d
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.utils import resample
from functools import reduce
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score

from sklearn import metrics, preprocessing
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, log_loss, recall_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, PowerTransformer

from sklearn.utils import check_consistent_length, column_or_1d, assert_all_finite
from sklearn.utils.extmath import stable_cumsum
from sklearn.utils.multiclass import type_of_target
from sklearn.linear_model import LogisticRegression
import re
import gc

import os

#from sklearn.externals import joblib
import joblib

from importlib import reload
import Filter_and_Grid_Search
Filter_and_Grid_Search = reload(Filter_and_Grid_Search)
from Filter_and_Grid_Search import stratified_split
from Filter_and_Grid_Search import attributes_list, attributes_list_new
from Filter_and_Grid_Search import get_s_stat, get_PSI_stat, get_stats_by_month, get_stats, stable_unstable
from Filter_and_Grid_Search import stable_unstable_by_month_divide, union_datas, individual_hists_all 
from Filter_and_Grid_Search import paired_time_hists_by_month, statistics_with_target
from Filter_and_Grid_Search import receive_correlations, find_doubles_corr
from Filter_and_Grid_Search import statistics_with_target, attributes_list, attributes_list_new, make_standard
from Filter_and_Grid_Search import data_preprocessing_train, data_preprocessing_test
from Filter_and_Grid_Search import receive_correlations, find_doubles_corr
from Filter_and_Grid_Search import stratified_split, two_forests, turn_variables_with_values
from Filter_and_Grid_Search import find_meta_params, calculate_vif#, find_meta_params_mem
from Filter_and_Grid_Search import plot_meta_2d, data_preprocessing, find_ouliers_iqr
from Filter_and_Grid_Search import train_model_receive_stats, simple_b_score_risk
from Filter_and_Grid_Search import max_prof_corve, by_month_gini, check_attribute_list_cases

from Filter_and_Grid_Search import to_zip, br_correction, br_stat

from statsmodels.stats.proportion import proportions_ztest

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import pathlib
import psutil

C:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
PATH_DATA = 'D:/Share/01 -Risk Desc Science/AntiFraud/Chinese_data/Datasets/'
PATH = 'D:/Share/safanasev/Python-notebook/AF_ML_chinese/'
print(PATH_DATA)
print(PATH)

D:/Share/01 -Risk Desc Science/AntiFraud/Chinese_data/Datasets/
D:/Share/safanasev/Python-notebook/AF_ML_chinese/


In [3]:
task = 'binary'
missing_strings = 'MISSING'
p_value = 0.05
target_dict = {'good': 0, 'bad': 1}

In [4]:
target = 'LABEL'
index_month = 'MONTH'
list_of_vars_for_strat = ['MONTH']
sort_by_var = 'ID'

necessary_fields = [target, index_month, sort_by_var]

COL_DEL = ['Unnamed: 0'] 
COL_DEL = [x.upper() for x in COL_DEL]
COL_TRG = target

In [5]:
n_jobs = 3

In [7]:
PATH + 'train_128_prep.csv'

'D:/Share/safanasev/Python-notebook/AF_ML_chinese/train_128_prep.csv'

## Loading preprocessed datasets

In [8]:
PATH = 'D:/Share/safanasev/Python-notebook/AF_ML_chinese/'
train_for = pd.read_csv(PATH + 'china_train_128_prep.csv')
valid_for = pd.read_csv(PATH + 'china_val_128_prep.csv')
test_for = pd.read_csv(PATH + 'china_test_128_prep.csv')

In [9]:
y_train = train_for[target]
y_val = valid_for[target]
y_test = test_for[target]

train_for.drop('Unnamed: 0', axis = 1, inplace = True)
valid_for.drop('Unnamed: 0', axis = 1, inplace = True)
test_for.drop('Unnamed: 0', axis = 1, inplace = True)


col = train_for.columns.to_list()
col.remove(target)


X_1_2 = train_for[col]
X_2_2 = valid_for[col]
X_3_2 = test_for[col]

In [10]:
X_2_2.columns

Index(['F1', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11',
       ...
       'F275', 'F276', 'F277', 'F283', 'F288', 'F296', 'F5_bin', 'F36_bin',
       'F48_bin', 'F166_bin'],
      dtype='object', length=128)

In [11]:
x_col = X_2_2.columns
y_col = 'LABEL'

In [13]:
from keras.utils import np_utils

In [29]:
Y_test_2 = np_utils.to_categorical( y_val, 2) # преобразовываем в 2 класса 

In [15]:
len(x_col)

128

In [16]:
train_for.shape

(792004, 129)

# Tensor

In [17]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Activation, Dense, Flatten, Dropout)
from tensorflow.keras.layers import (Conv1D, MaxPooling1D, AveragePooling1D)
from tensorflow.keras.layers import add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from keras.utils import np_utils

In [18]:
from tensorflow.keras.layers import ReLU, Concatenate, GlobalAveragePooling1D

In [19]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 29

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.compat.v1.set_random_seed(seed_value)


# 5. Configure a new global `tensorflow` session
# for later versions:
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [20]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7645518600221670866
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 6375109240958848568
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 13333621266457161539
physical_device_desc: "device: XLA_GPU device"
]


In [21]:
#если alpha не задано, то не делаем br_correction

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self,  dataset, x_col, y_col,  batch_size=256, num_classes=2, alpha = 0.1, random_state = 42, 
                 shuffle=True, class_w = None):
        self.batch_size = batch_size
        self.dataset = dataset
        self.x_col = x_col
        self.y_col = y_col
        self.indices = self.dataset.index.tolist()
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.alpha = alpha
        self.random_state = random_state
        self.class_w = class_w
        self.on_epoch_end()

    def __len__(self):
        return math.ceil(len(self.indices) / self.batch_size)

    def __getitem__(self, index):
        index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in index]
        
        data_tmp = self.dataset.loc[batch]
        data_tmp_b = data_tmp[data_tmp[self.y_col] == 1]
        data_tmp_b_all = self.dataset[self.dataset[self.y_col] == 1]
        X_tmp = data_tmp.head(0)
        
        
        #print(data_tmp_b.shape[0], data_tmp.shape[0], data_tmp_b_all.shape[0])
        if (self.alpha is None) and (self.class_w is None):
            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col] 
            
        elif (self.alpha is None) and self.class_w > 0:
            k = (self.class_w*self.batch_size)/(1-self.class_w)
            k = k - data_tmp_b.shape[0]
            if int(k) < 1:
                k = k+1
            ix2 = np.random.RandomState(self.random_state + int(k)).choice(data_tmp_b_all.shape[0], int(k))
            X_tmp = data_tmp_b_all.iloc[ix2]
            data_tmp = data_tmp.append(X_tmp, ignore_index=True)
            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col]
            #print(data_tmp_b.shape[0], data_tmp.shape[0],  data_tmp_b.shape[0]/(data_tmp.shape[0] - X_tmp.shape[0]), np.mean(y)) 
            
            
            
        else:
            if data_tmp_b.shape[0] > 0:
                k=(self.alpha*self.batch_size/data_tmp_b.shape[0] -1 )/(1-self.alpha)
                for i in range(0,int(k)):
                    X_tmp = X_tmp.append(data_tmp_b, ignore_index=True)
            else:
                k = (self.alpha*self.batch_size)/(1-self.alpha)
                ix2 = np.random.RandomState(self.random_state + int(k)).choice(data_tmp_b_all.shape[0], int(k))
                X_tmp = data_tmp_b_all.iloc[ix2]
                data_tmp_b = X_tmp

    #         print('Добавим дробное число строк')    
            k_fraction = k - int(k)
            n_samples = int(round(data_tmp_b.shape[0]*k_fraction))
    #         print(k_fraction, n_samples)

            ix = np.random.RandomState(self.random_state).choice(data_tmp_b.shape[0], n_samples)
            data_add_fraction = data_tmp_b.iloc[ix]
            X_tmp.append(data_add_fraction, ignore_index=True)

            data_tmp = data_tmp.append(X_tmp, ignore_index=True)


            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col]   
        #print(sum(self.dataset.loc[batch][self.y_col])/len(index), np.mean(y))
        
        if self.num_classes > 1:
            y = np_utils.to_categorical( y, self.num_classes)
            
        ####for tf 2.3.0, [np.array(X)], np.array(y)

        #return [np.array(X).reshape(X.shape[0], X.shape[1], 1)], np.array(y)
        return [np.array(X)], np.array(y)

    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)

In [22]:
from tensorflow.keras.callbacks import EarlyStopping, Callback

# объявляем класс метрик

class E_time(Callback):
    def on_train_begin(self, logs={}):
        tm = datetime.strftime(datetime.now(), "%d.%m.%Y %H:%M:%S")
        print ('train_begin', '| time: ' , tm)
        
    def on_epoch_end(self, epoch, logs={}):
        tm = datetime.strftime(datetime.now(), "%d.%m.%Y %H:%M:%S")
        print ('epoch_end', '| time: ' , tm)
    
        return

_time = E_time()

In [23]:
class Metrics(tf.keras.callbacks.Callback):
    def __init__(self, valid_data):
        super(Metrics, self).__init__()
        self.validation_data = valid_data

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        val_predict = self.model.predict(self.validation_data[0])[:, 1]
        val_targ = self.validation_data[1]
        
        if len(val_targ.shape) == 2: #and val_targ.shape[1] != 1:
            val_targ = val_targ[:,1]

        _val_aps = metrics.average_precision_score(val_targ, val_predict)
        #_val_recall = metrics.recall_score(val_targ, val_predict)
        _val_a = metrics.roc_auc_score(val_targ, val_predict)

        logs['val_aps'] = _val_aps
        logs['val_a'] = _val_a
        print(" — val_aps:  %f — val_a: %f" % (_val_aps, _val_a))
        return

In [24]:
def max_pool(min_pool, n_pool, padding_pool,  str_pool, y, str_min = 1):
    #for 3D-tensor
    if y.shape[1] < (min_pool):
        
        return MaxPooling1D(pool_size = n_pool, padding=padding_pool, strides=str_min)(y)
    else:
        return MaxPooling1D(pool_size = n_pool, padding=padding_pool, strides=str_pool)(y)

## CNN

In [25]:
def model_cnn_3(n_features=163, n_pool=2, n_kernel=5, n_filters=5,
                n_strides=1, n_strides_pool=None, classes=2, hidden=64, drop_out=0.25,
                l2_conv=None, reg=1E-5, reg_dense=1E-5, _bias=True, gl_pool_max=False, padding_pool='valid',
                padding_conv = 'same', n_lay=3, min_pool = 15):

    if reg == None:
        l2_batch_gamma = None
        l2_batch_betta = None
    else:
        l2_batch_gamma = l2(reg)
        l2_batch_betta = l2(reg)

    if reg_dense == None:
        kernel_regularizer = None
    else:
        kernel_regularizer = l2(reg)

    # 1 слой

    x = Input(shape=(n_features, 1))
    y = Conv1D(filters=n_filters, kernel_size=n_kernel,
               strides=n_strides, padding=padding_conv, use_bias=_bias)(x)

    y = BatchNormalization(gamma_regularizer=l2_batch_gamma,
                           beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)
    y = max_pool(min_pool, n_pool, padding_pool, n_strides_pool, y)

    for i in range(1, n_lay):
        y = Conv1D(filters=(i + 1)* n_filters, kernel_size=n_kernel,
                   strides=n_strides, padding=padding_conv, use_bias=_bias)(y)
        y = BatchNormalization(gamma_regularizer=l2_batch_gamma,
                           beta_regularizer=l2_batch_betta)(y)
        y = ReLU()(y)
        y = max_pool(min_pool, n_pool, padding_pool, n_strides_pool, y)
    if gl_pool_max:
        z = GlobalMaxPooling1D()(y)
        
    else:
        z = GlobalAveragePooling1D()(y)
        
    z = Dense(hidden, activation='relu', kernel_regularizer = kernel_regularizer)(z)
    z = Dropout(drop_out)(z)
    z = Dense(hidden, activation='relu', kernel_regularizer = kernel_regularizer)(z)
    z = Dropout(drop_out)(z)
    predictions = Dense(classes, activation='softmax')(z)

    model = Model(inputs=x, outputs=predictions)


    return model

## GridSearch and Training CNN-3 and CNN-6

In [26]:
gl_p = [False, True] #0
l2_batch = [ 0.0002] #1
n_ker = [5, 7] #2
n_fil = [ 5, 15] #3
d_hidden = [60, 100] #4
drop_out = [0.25] #5
reg_dense = [ 0.0002] #6
min_pool = [ 20] #7
n_lay = [3, 6] #8, CNN-3 and CNN-6

In [27]:
param = itertools.product(gl_p, l2_batch, n_ker, n_fil, d_hidden, drop_out, reg_dense, min_pool, n_lay)

In [28]:
inp_shape = X_2_2.shape[1]
inp_shape

128

In [30]:
training_generator = DataGenerator(train_for, x_col, y_col, batch_size=512, alpha = None, class_w = 0.1 )

In [31]:
VERBOSE = 2
BATCH_SIZE = 512
NB_EPOCH = 150
class_weighting = {0: 0.1, 1: 1}

result_all_8 = pd.DataFrame()
result_all_8['name_model'] = None
result_all_8['params'] = None
result_all_8['val_GINI'] = None
result_all_8['val_APS'] = None
result_all_8['test_GINI'] = None
result_all_8['test_APS'] = None
j = 0

for p in param:
    print('=====================================================================================================')
    print(inp_shape, X_2_2.shape, p)

    model_grid = model_cnn_3(reg=p[1], reg_dense=p[6], n_features=inp_shape, n_pool=2, n_kernel=p[2],
                             n_filters=p[3], n_strides=1, classes=2,
                             hidden=p[4], drop_out=p[5],
                             gl_pool_max=p[0], min_pool=p[7], n_lay=p[8])

    model_grid.compile(loss='categorical_crossentropy',
                       optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
                       metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    history_XX = model_grid.fit_generator(generator=training_generator,  # training_aug,
                                          # Y_test_2 = np_utils.to_categorical( y_val, 2)
                                          validation_data=(X_2_2, Y_test_2),
                                          epochs=NB_EPOCH, verbose=VERBOSE, class_weight=class_weighting,
                                          callbacks=[_time, EarlyStopping(monitor='val_loss', patience=20),
                                                     Metrics(valid_data=(X_2_2, Y_test_2))])

    res_model_ = pd.DataFrame(
        history_XX.history, columns=history_XX.history.keys())
    dd = str(200000 + datetime.now().month*100 +
             datetime.now().day) + '_' + str(datetime.now().hour)
    name_m = 'model_cnn_china_grid_' + str(p[0]) + '_' + str(
        p[2]) + '_' + str(p[3]) + '_' + str(p[4]) + '_' + str(p[8]) + '_' + str(dd)

    model_grid.save(name_m + '.h5')
    res_model_.to_csv(name_m + '.csv')

    predict_class_val = model_grid.predict(X_2_2)
    APS = metrics.average_precision_score(y_val, predict_class_val[:,1])
    GINI = 2*(metrics.roc_auc_score(y_val , predict_class_val[:,1])) - 1

    predict_class_test = model_grid.predict(X_3_2)
    APS_t = metrics.average_precision_score(y_test, predict_class_test[:,1])
    GINI_t = 2*(metrics.roc_auc_score(y_test, predict_class_test[:,1])) - 1

    result_all_8.at[j, 'name_model'] = name_m
    result_all_8.at[j, 'params'] = str(p)
    result_all_8.at[j, 'val_GINI'] = GINI
    result_all_8.at[j, 'val_APS'] = APS
    result_all_8.at[j, 'test_GINI'] = GINI_t
    result_all_8.at[j, 'test_APS'] = APS_t

    result_all_8.to_csv('model_cnn_china_grid_all.csv')
    j += 1
    print(name_m, 'valid_for_train: ', GINI, APS, '| test: ', GINI_t, APS_t)

128 (99001, 128) (False, 0.0002, 5, 5, 60, 0.25, 0.0002, 20, 3)
Instructions for updating:
Please use Model.fit, which supports generators.
train_begin | time:  27.04.2021 19:42:37
Epoch 1/150
epoch_end | time:  27.04.2021 19:48:43
 — val_aps:  0.306260 — val_a: 0.943905
1547/1547 - 390s - loss: 0.0519 - accuracy: 0.8922 - auc: 0.9620 - precision: 0.8922 - recall: 0.8922 - val_loss: 0.5184 - val_accuracy: 0.7959 - val_auc: 0.8944 - val_precision: 0.7959 - val_recall: 0.7959
Epoch 2/150
epoch_end | time:  27.04.2021 19:55:45
 — val_aps:  0.324590 — val_a: 0.946153
1547/1547 - 425s - loss: 0.0392 - accuracy: 0.9270 - auc: 0.9773 - precision: 0.9270 - recall: 0.9270 - val_loss: 0.0638 - val_accuracy: 0.9841 - val_auc: 0.9975 - val_precision: 0.9841 - val_recall: 0.9841
Epoch 3/150
epoch_end | time:  27.04.2021 20:02:10
 — val_aps:  0.335289 — val_a: 0.948796
1547/1547 - 383s - loss: 0.0355 - accuracy: 0.9368 - auc: 0.9806 - precision: 0.9368 - recall: 0.9368 - val_loss: 0.1704 - val_accur

Epoch 27/150
epoch_end | time:  27.04.2021 22:38:19
 — val_aps:  0.365513 — val_a: 0.953513
1547/1547 - 388s - loss: 0.0265 - accuracy: 0.9622 - auc: 0.9857 - precision: 0.9622 - recall: 0.9622 - val_loss: 0.1129 - val_accuracy: 0.9685 - val_auc: 0.9905 - val_precision: 0.9685 - val_recall: 0.9685
model_cnn_china_grid_False_5_5_60_3_200427_22 valid_for_train:  0.9070262986580488 0.36551338173392156 | test:  0.903199798959144 0.3660026242855116
128 (99001, 128) (False, 0.0002, 5, 5, 60, 0.25, 0.0002, 20, 6)
train_begin | time:  27.04.2021 22:39:13
Epoch 1/150
epoch_end | time:  27.04.2021 22:46:40
 — val_aps:  0.355385 — val_a: 0.954768
1547/1547 - 474s - loss: 0.0404 - accuracy: 0.9421 - auc_1: 0.9820 - precision_1: 0.9421 - recall_1: 0.9421 - val_loss: 0.0819 - val_accuracy: 0.9752 - val_auc_1: 0.9962 - val_precision_1: 0.9752 - val_recall_1: 0.9752
Epoch 2/150
epoch_end | time:  27.04.2021 22:53:51
 — val_aps:  0.406155 — val_a: 0.962544
1547/1547 - 431s - loss: 0.0242 - accuracy: 0.

Epoch 26/150
epoch_end | time:  28.04.2021 01:55:50
 — val_aps:  0.433364 — val_a: 0.968531
1547/1547 - 484s - loss: 0.0148 - accuracy: 0.9773 - auc_1: 0.9955 - precision_1: 0.9773 - recall_1: 0.9773 - val_loss: 0.0782 - val_accuracy: 0.9783 - val_auc_1: 0.9949 - val_precision_1: 0.9783 - val_recall_1: 0.9783
Epoch 27/150
epoch_end | time:  28.04.2021 02:03:13
 — val_aps:  0.423864 — val_a: 0.966456
1547/1547 - 443s - loss: 0.0148 - accuracy: 0.9776 - auc_1: 0.9955 - precision_1: 0.9776 - recall_1: 0.9776 - val_loss: 0.0890 - val_accuracy: 0.9738 - val_auc_1: 0.9936 - val_precision_1: 0.9738 - val_recall_1: 0.9738
Epoch 28/150
epoch_end | time:  28.04.2021 02:11:09
 — val_aps:  0.437974 — val_a: 0.968908
1547/1547 - 475s - loss: 0.0146 - accuracy: 0.9780 - auc_1: 0.9957 - precision_1: 0.9780 - recall_1: 0.9780 - val_loss: 0.0747 - val_accuracy: 0.9774 - val_auc_1: 0.9951 - val_precision_1: 0.9774 - val_recall_1: 0.9774
Epoch 29/150
epoch_end | time:  28.04.2021 02:18:42
 — val_aps:  0.

Epoch 19/150
epoch_end | time:  28.04.2021 04:50:19
 — val_aps:  0.352859 — val_a: 0.947718
1547/1547 - 256s - loss: 0.0269 - accuracy: 0.9609 - auc_2: 0.9857 - precision_2: 0.9609 - recall_2: 0.9609 - val_loss: 0.1280 - val_accuracy: 0.9626 - val_auc_2: 0.9872 - val_precision_2: 0.9626 - val_recall_2: 0.9626
Epoch 20/150
epoch_end | time:  28.04.2021 04:54:31
 — val_aps:  0.346269 — val_a: 0.945247
1547/1547 - 257s - loss: 0.0274 - accuracy: 0.9599 - auc_2: 0.9849 - precision_2: 0.9599 - recall_2: 0.9599 - val_loss: 0.1001 - val_accuracy: 0.9703 - val_auc_2: 0.9934 - val_precision_2: 0.9703 - val_recall_2: 0.9703
Epoch 21/150
epoch_end | time:  28.04.2021 04:59:07
 — val_aps:  0.342270 — val_a: 0.944375
1547/1547 - 275s - loss: 0.0267 - accuracy: 0.9611 - auc_2: 0.9854 - precision_2: 0.9611 - recall_2: 0.9611 - val_loss: 0.1278 - val_accuracy: 0.9599 - val_auc_2: 0.9894 - val_precision_2: 0.9599 - val_recall_2: 0.9599
Epoch 22/150
epoch_end | time:  28.04.2021 05:03:33
 — val_aps:  0.

Epoch 9/150
epoch_end | time:  28.04.2021 06:55:12
 — val_aps:  0.448205 — val_a: 0.968084
1547/1547 - 343s - loss: 0.0172 - accuracy: 0.9751 - auc_3: 0.9936 - precision_3: 0.9751 - recall_3: 0.9751 - val_loss: 0.0858 - val_accuracy: 0.9766 - val_auc_3: 0.9935 - val_precision_3: 0.9766 - val_recall_3: 0.9766
Epoch 10/150
epoch_end | time:  28.04.2021 07:00:43
 — val_aps:  0.408209 — val_a: 0.964181
1547/1547 - 332s - loss: 0.0169 - accuracy: 0.9753 - auc_3: 0.9937 - precision_3: 0.9753 - recall_3: 0.9753 - val_loss: 0.0983 - val_accuracy: 0.9747 - val_auc_3: 0.9923 - val_precision_3: 0.9747 - val_recall_3: 0.9747
Epoch 11/150
epoch_end | time:  28.04.2021 07:06:26
 — val_aps:  0.377682 — val_a: 0.962251
1547/1547 - 342s - loss: 0.0165 - accuracy: 0.9756 - auc_3: 0.9940 - precision_3: 0.9756 - recall_3: 0.9756 - val_loss: 0.2271 - val_accuracy: 0.9333 - val_auc_3: 0.9707 - val_precision_3: 0.9333 - val_recall_3: 0.9333
Epoch 12/150
epoch_end | time:  28.04.2021 07:12:12
 — val_aps:  0.4

...

Epoch 63/150
epoch_end | time:  28.04.2021 12:38:36
 — val_aps:  0.443897 — val_a: 0.964871
1547/1547 - 336s - loss: 0.0134 - accuracy: 0.9789 - auc_3: 0.9963 - precision_3: 0.9789 - recall_3: 0.9789 - val_loss: 0.0782 - val_accuracy: 0.9759 - val_auc_3: 0.9950 - val_precision_3: 0.9759 - val_recall_3: 0.9759
Epoch 64/150
epoch_end | time:  28.04.2021 12:43:54
 — val_aps:  0.447468 — val_a: 0.965971
1547/1547 - 318s - loss: 0.0134 - accuracy: 0.9793 - auc_3: 0.9963 - precision_3: 0.9793 - recall_3: 0.9793 - val_loss: 0.0718 - val_accuracy: 0.9759 - val_auc_3: 0.9958 - val_precision_3: 0.9759 - val_recall_3: 0.9759
Epoch 65/150
epoch_end | time:  28.04.2021 12:49:29
 — val_aps:  0.465813 — val_a: 0.966880
1547/1547 - 333s - loss: 0.0132 - accuracy: 0.9795 - auc_3: 0.9964 - precision_3: 0.9795 - recall_3: 0.9795 - val_loss: 0.0763 - val_accuracy: 0.9758 - val_auc_3: 0.9950 - val_precision_3: 0.9758 - val_recall_3: 0.9758
Epoch 66/150
epoch_end | time:  28.04.2021 12:55:16
 — val_aps:  0.

Epoch 23/150
epoch_end | time:  28.04.2021 16:20:52
 — val_aps:  0.428655 — val_a: 0.968050
1547/1547 - 532s - loss: 0.0175 - accuracy: 0.9772 - auc_4: 0.9935 - precision_4: 0.9772 - recall_4: 0.9772 - val_loss: 0.1255 - val_accuracy: 0.9619 - val_auc_4: 0.9901 - val_precision_4: 0.9619 - val_recall_4: 0.9619
Epoch 24/150
epoch_end | time:  28.04.2021 16:30:10
 — val_aps:  0.443667 — val_a: 0.969912
1547/1547 - 567s - loss: 0.0172 - accuracy: 0.9780 - auc_4: 0.9937 - precision_4: 0.9780 - recall_4: 0.9780 - val_loss: 0.1015 - val_accuracy: 0.9708 - val_auc_4: 0.9932 - val_precision_4: 0.9708 - val_recall_4: 0.9708
Epoch 25/150
epoch_end | time:  28.04.2021 16:40:11
 — val_aps:  0.468757 — val_a: 0.969535
1547/1547 - 596s - loss: 0.0172 - accuracy: 0.9784 - auc_4: 0.9937 - precision_4: 0.9784 - recall_4: 0.9784 - val_loss: 0.0725 - val_accuracy: 0.9820 - val_auc_4: 0.9939 - val_precision_4: 0.9820 - val_recall_4: 0.9820
Epoch 26/150
epoch_end | time:  28.04.2021 16:52:30
 — val_aps:  0.

 — val_aps:  0.460067 — val_a: 0.971292
1547/1547 - 1862s - loss: 0.0112 - accuracy: 0.9826 - auc_5: 0.9976 - precision_5: 0.9826 - recall_5: 0.9826 - val_loss: 0.1034 - val_accuracy: 0.9670 - val_auc_5: 0.9926 - val_precision_5: 0.9670 - val_recall_5: 0.9670
Epoch 17/150
epoch_end | time:  29.04.2021 03:29:44
 — val_aps:  0.447090 — val_a: 0.969953
1547/1547 - 1910s - loss: 0.0111 - accuracy: 0.9824 - auc_5: 0.9977 - precision_5: 0.9824 - recall_5: 0.9824 - val_loss: 0.0615 - val_accuracy: 0.9812 - val_auc_5: 0.9968 - val_precision_5: 0.9812 - val_recall_5: 0.9812
Epoch 18/150
epoch_end | time:  29.04.2021 04:00:58
 — val_aps:  0.494976 — val_a: 0.972980
1547/1547 - 1858s - loss: 0.0110 - accuracy: 0.9826 - auc_5: 0.9977 - precision_5: 0.9826 - recall_5: 0.9826 - val_loss: 0.0860 - val_accuracy: 0.9756 - val_auc_5: 0.9945 - val_precision_5: 0.9756 - val_recall_5: 0.9756
Epoch 19/150
epoch_end | time:  29.04.2021 04:29:48
 — val_aps:  0.484602 — val_a: 0.970056
1547/1547 - 1720s - loss

Epoch 43/150
epoch_end | time:  29.04.2021 16:06:35
 — val_aps:  0.431537 — val_a: 0.966436
1547/1547 - 1377s - loss: 0.0092 - accuracy: 0.9855 - auc_5: 0.9984 - precision_5: 0.9855 - recall_5: 0.9855 - val_loss: 0.0701 - val_accuracy: 0.9793 - val_auc_5: 0.9957 - val_precision_5: 0.9793 - val_recall_5: 0.9793
Epoch 44/150
epoch_end | time:  29.04.2021 16:28:49
 — val_aps:  0.442989 — val_a: 0.967073
1547/1547 - 1316s - loss: 0.0090 - accuracy: 0.9858 - auc_5: 0.9984 - precision_5: 0.9858 - recall_5: 0.9858 - val_loss: 0.0676 - val_accuracy: 0.9791 - val_auc_5: 0.9961 - val_precision_5: 0.9791 - val_recall_5: 0.9791
Epoch 45/150
epoch_end | time:  29.04.2021 16:44:29
 — val_aps:  0.409606 — val_a: 0.960741
1547/1547 - 939s - loss: 0.0092 - accuracy: 0.9852 - auc_5: 0.9983 - precision_5: 0.9852 - recall_5: 0.9852 - val_loss: 0.0599 - val_accuracy: 0.9857 - val_auc_5: 0.9962 - val_precision_5: 0.9857 - val_recall_5: 0.9857
Epoch 46/150
epoch_end | time:  29.04.2021 17:00:10
 — val_aps:  

KeyboardInterrupt: 

In [32]:
result_all_8

,name_model,params,val_GINI,val_APS,test_GINI,test_APS
0,model_cnn_china_grid_False_5_5_60_3_200427_22,"(False, 0.0002, 5, 5, 60, 0.25, 0.0002, 20, 3)",0.907026,0.365513,0.9032,0.366003
1,model_cnn_china_grid_False_5_5_60_6_200428_2,"(False, 0.0002, 5, 5, 60, 0.25, 0.0002, 20, 6)",0.940143,0.463882,0.93795,0.478185
2,model_cnn_china_grid_False_5_5_100_3_200428_6,"(False, 0.0002, 5, 5, 100, 0.25, 0.0002, 20, 3)",0.89876,0.322023,0.89824,0.322231
3,model_cnn_china_grid_False_5_5_100_6_200428_12,"(False, 0.0002, 5, 5, 100, 0.25, 0.0002, 20, 6)",0.930077,0.460635,0.942235,0.452138
4,model_cnn_china_grid_False_5_15_60_3_200428_18,"(False, 0.0002, 5, 15, 60, 0.25, 0.0002, 20, 3)",0.937533,0.458496,0.933944,0.446152
5,model_cnn_china_grid_False_5_15_60_6_200429_19,"(False, 0.0002, 5, 15, 60, 0.25, 0.0002, 20, 6)",0.933843,0.483434,0.935933,0.490821


## GridSearch and Training CNN-6 and CNN-8

In [34]:
gl_p = [False, True] #0
l2_batch = [ 0.0001 ] #1
n_ker = [3, 5] #2
n_fil = [ 5, 10] #3
d_hidden = [30, 100] #4
drop_out = [0.25] #5
reg_dense = [ 0.0002] #6
min_pool = [ 20] #7
n_lay = [6, 8] #8, CNN-6 and CNN-8

In [35]:
param = itertools.product(gl_p, l2_batch, n_ker, n_fil, d_hidden, drop_out, reg_dense, min_pool, n_lay)

In [36]:
k = 0
for i in param:
    print(k, i)
    k +=1

0 (False, 0.0001, 3, 5, 30, 0.25, 0.0002, 20, 6)
1 (False, 0.0001, 3, 5, 30, 0.25, 0.0002, 20, 8)
2 (False, 0.0001, 3, 5, 100, 0.25, 0.0002, 20, 6)
3 (False, 0.0001, 3, 5, 100, 0.25, 0.0002, 20, 8)
4 (False, 0.0001, 3, 10, 30, 0.25, 0.0002, 20, 6)
5 (False, 0.0001, 3, 10, 30, 0.25, 0.0002, 20, 8)
6 (False, 0.0001, 3, 10, 100, 0.25, 0.0002, 20, 6)
7 (False, 0.0001, 3, 10, 100, 0.25, 0.0002, 20, 8)
8 (False, 0.0001, 5, 5, 30, 0.25, 0.0002, 20, 6)
9 (False, 0.0001, 5, 5, 30, 0.25, 0.0002, 20, 8)
10 (False, 0.0001, 5, 5, 100, 0.25, 0.0002, 20, 6)
11 (False, 0.0001, 5, 5, 100, 0.25, 0.0002, 20, 8)
12 (False, 0.0001, 5, 10, 30, 0.25, 0.0002, 20, 6)
13 (False, 0.0001, 5, 10, 30, 0.25, 0.0002, 20, 8)
14 (False, 0.0001, 5, 10, 100, 0.25, 0.0002, 20, 6)
15 (False, 0.0001, 5, 10, 100, 0.25, 0.0002, 20, 8)
16 (True, 0.0001, 3, 5, 30, 0.25, 0.0002, 20, 6)
17 (True, 0.0001, 3, 5, 30, 0.25, 0.0002, 20, 8)
18 (True, 0.0001, 3, 5, 100, 0.25, 0.0002, 20, 6)
19 (True, 0.0001, 3, 5, 100, 0.25, 0.0002, 20,

In [37]:
param = ((False, 0.0001, 5, 5, 30, 0.25, 0.0002, 20, 6), 
         (False, 0.0001, 5, 5, 30, 0.25, 0.0002, 20, 8),
        (False, 0.0001, 5, 10, 30, 0.25, 0.0002, 20, 6),
         (False, 0.0001, 5, 10, 30, 0.25, 0.0002, 20, 8),
        (False, 0.0001, 3, 10, 100, 0.25, 0.0002, 20, 6),
         (False, 0.0001, 3, 10, 100, 0.25, 0.0002, 20, 8),
        (False, 0.0002, 5, 15, 60, 0.25, 0.0002, 20, 8))

In [38]:
VERBOSE = 2
BATCH_SIZE = 512
NB_EPOCH = 150
class_weighting = {0: 0.1, 1: 1}

result_all_8 = pd.DataFrame()
result_all_8['name_model'] = None
result_all_8['params'] = None
result_all_8['val_GINI'] = None
result_all_8['val_APS'] = None
result_all_8['test_GINI'] = None
result_all_8['test_APS'] = None
j = 0

for p in param:
    print('=====================================================================================================')
    print(inp_shape, X_2_2.shape, p)

    model_grid = model_cnn_3(reg=p[1], reg_dense=p[6], n_features=inp_shape, n_pool=2, n_kernel=p[2],
                             n_filters=p[3], n_strides=1, classes=2,
                             hidden=p[4], drop_out=p[5],
                             gl_pool_max=p[0], min_pool=p[7], n_lay=p[8])

    model_grid.compile(loss='categorical_crossentropy',
                       optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
                       metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    history_XX = model_grid.fit_generator(generator=training_generator,  # training_aug,
                                          # Y_test_2 = np_utils.to_categorical( y_val, 2)
                                          validation_data=(X_2_2, Y_test_2),
                                          epochs=NB_EPOCH, verbose=VERBOSE, class_weight=class_weighting,
                                          callbacks=[_time, EarlyStopping(monitor='val_loss', patience=20),
                                                     Metrics(valid_data=(X_2_2, Y_test_2))])

    res_model_ = pd.DataFrame(
        history_XX.history, columns=history_XX.history.keys())
    dd = str(200000 + datetime.now().month*100 +
             datetime.now().day) + '_' + str(datetime.now().hour)
    name_m = 'model_cnn_china_grid_' + str(p[0]) + '_' + str(
        p[2]) + '_' + str(p[3]) + '_' + str(p[4]) + '_' + str(p[8]) + '_' + str(dd)

    model_grid.save(name_m + '.h5')
    res_model_.to_csv(name_m + '.csv')

    predict_class_val = model_grid.predict(X_2_2)
    APS = metrics.average_precision_score(y_val, predict_class_val[:,1])
    GINI = 2*(metrics.roc_auc_score(y_val , predict_class_val[:,1])) - 1

    predict_class_test = model_grid.predict(X_3_2)
    APS_t = metrics.average_precision_score(y_test, predict_class_test[:,1])
    GINI_t = 2*(metrics.roc_auc_score(y_test, predict_class_test[:,1])) - 1

    result_all_8.at[j, 'name_model'] = name_m
    result_all_8.at[j, 'params'] = str(p)
    result_all_8.at[j, 'val_GINI'] = GINI
    result_all_8.at[j, 'val_APS'] = APS
    result_all_8.at[j, 'test_GINI'] = GINI_t
    result_all_8.at[j, 'test_APS'] = APS_t

    result_all_8.to_csv('model_cnn_china_grid_all.csv')
    j += 1
    print(name_m, 'valid_for_train: ', GINI, APS, '| test: ', GINI_t, APS_t)

128 (99001, 128) (False, 0.0001, 5, 5, 30, 0.25, 0.0002, 20, 6)
train_begin | time:  29.04.2021 21:20:38
Epoch 1/150
epoch_end | time:  29.04.2021 21:26:05
 — val_aps:  0.362349 — val_a: 0.958090
1547/1547 - 339s - loss: 0.0381 - accuracy: 0.9403 - auc_7: 0.9812 - precision_7: 0.9403 - recall_7: 0.9403 - val_loss: 0.1016 - val_accuracy: 0.9695 - val_auc_7: 0.9935 - val_precision_7: 0.9695 - val_recall_7: 0.9695
Epoch 2/150
epoch_end | time:  29.04.2021 21:31:46
 — val_aps:  0.372410 — val_a: 0.962142
1547/1547 - 340s - loss: 0.0239 - accuracy: 0.9666 - auc_7: 0.9898 - precision_7: 0.9666 - recall_7: 0.9666 - val_loss: 0.1136 - val_accuracy: 0.9681 - val_auc_7: 0.9903 - val_precision_7: 0.9681 - val_recall_7: 0.9681
Epoch 3/150
epoch_end | time:  29.04.2021 21:37:12
 — val_aps:  0.384463 — val_a: 0.962306
1547/1547 - 328s - loss: 0.0210 - accuracy: 0.9705 - auc_7: 0.9912 - precision_7: 0.9705 - recall_7: 0.9705 - val_loss: 0.0952 - val_accuracy: 0.9726 - val_auc_7: 0.9929 - val_precisio

Epoch 27/150
epoch_end | time:  29.04.2021 23:47:46
 — val_aps:  0.428896 — val_a: 0.968083
1547/1547 - 312s - loss: 0.0142 - accuracy: 0.9771 - auc_7: 0.9958 - precision_7: 0.9771 - recall_7: 0.9771 - val_loss: 0.1514 - val_accuracy: 0.9471 - val_auc_7: 0.9853 - val_precision_7: 0.9471 - val_recall_7: 0.9471
Epoch 28/150
epoch_end | time:  29.04.2021 23:53:07
 — val_aps:  0.458113 — val_a: 0.969856
1547/1547 - 320s - loss: 0.0140 - accuracy: 0.9778 - auc_7: 0.9960 - precision_7: 0.9778 - recall_7: 0.9778 - val_loss: 0.0716 - val_accuracy: 0.9775 - val_auc_7: 0.9956 - val_precision_7: 0.9775 - val_recall_7: 0.9775
Epoch 29/150
epoch_end | time:  29.04.2021 23:58:28
 — val_aps:  0.455182 — val_a: 0.970280
1547/1547 - 320s - loss: 0.0138 - accuracy: 0.9779 - auc_7: 0.9960 - precision_7: 0.9779 - recall_7: 0.9779 - val_loss: 0.0708 - val_accuracy: 0.9746 - val_auc_7: 0.9962 - val_precision_7: 0.9746 - val_recall_7: 0.9746
Epoch 30/150
epoch_end | time:  30.04.2021 00:03:42
 — val_aps:  0.

Epoch 15/150
epoch_end | time:  30.04.2021 02:30:04
 — val_aps:  0.366450 — val_a: 0.967027
1547/1547 - 408s - loss: 0.0140 - accuracy: 0.9788 - auc_8: 0.9957 - precision_8: 0.9788 - recall_8: 0.9788 - val_loss: 0.1162 - val_accuracy: 0.9669 - val_auc_8: 0.9896 - val_precision_8: 0.9669 - val_recall_8: 0.9669
Epoch 16/150
epoch_end | time:  30.04.2021 02:36:59
 — val_aps:  0.450978 — val_a: 0.969302
1547/1547 - 415s - loss: 0.0140 - accuracy: 0.9789 - auc_8: 0.9958 - precision_8: 0.9789 - recall_8: 0.9789 - val_loss: 0.0878 - val_accuracy: 0.9721 - val_auc_8: 0.9939 - val_precision_8: 0.9721 - val_recall_8: 0.9721
Epoch 17/150
epoch_end | time:  30.04.2021 02:43:47
 — val_aps:  0.405427 — val_a: 0.968956
1547/1547 - 406s - loss: 0.0138 - accuracy: 0.9792 - auc_8: 0.9959 - precision_8: 0.9792 - recall_8: 0.9792 - val_loss: 0.1207 - val_accuracy: 0.9569 - val_auc_8: 0.9904 - val_precision_8: 0.9569 - val_recall_8: 0.9569
Epoch 18/150
epoch_end | time:  30.04.2021 02:50:46
 — val_aps:  0.

Epoch 13/150
epoch_end | time:  30.04.2021 06:20:27
 — val_aps:  0.483836 — val_a: 0.973455
1547/1547 - 642s - loss: 0.0123 - accuracy: 0.9799 - auc_9: 0.9968 - precision_9: 0.9799 - recall_9: 0.9799 - val_loss: 0.0544 - val_accuracy: 0.9825 - val_auc_9: 0.9971 - val_precision_9: 0.9825 - val_recall_9: 0.9825
Epoch 14/150
epoch_end | time:  30.04.2021 06:31:24
 — val_aps:  0.478439 — val_a: 0.973418
1547/1547 - 657s - loss: 0.0121 - accuracy: 0.9798 - auc_9: 0.9970 - precision_9: 0.9798 - recall_9: 0.9798 - val_loss: 0.0507 - val_accuracy: 0.9812 - val_auc_9: 0.9980 - val_precision_9: 0.9812 - val_recall_9: 0.9812
Epoch 15/150
epoch_end | time:  30.04.2021 06:42:33
 — val_aps:  0.468779 — val_a: 0.970516
1547/1547 - 668s - loss: 0.0120 - accuracy: 0.9804 - auc_9: 0.9970 - precision_9: 0.9804 - recall_9: 0.9804 - val_loss: 0.0527 - val_accuracy: 0.9845 - val_auc_9: 0.9972 - val_precision_9: 0.9845 - val_recall_9: 0.9845
Epoch 16/150
epoch_end | time:  30.04.2021 06:53:21
 — val_aps:  0.

Epoch 40/150
epoch_end | time:  30.04.2021 11:13:45
 — val_aps:  0.328981 — val_a: 0.961475
1547/1547 - 642s - loss: 0.0097 - accuracy: 0.9832 - auc_9: 0.9981 - precision_9: 0.9832 - recall_9: 0.9832 - val_loss: 0.0779 - val_accuracy: 0.9734 - val_auc_9: 0.9954 - val_precision_9: 0.9734 - val_recall_9: 0.9734
Epoch 41/150
epoch_end | time:  30.04.2021 11:27:20
 — val_aps:  0.471394 — val_a: 0.966479
1547/1547 - 828s - loss: 0.0098 - accuracy: 0.9827 - auc_9: 0.9980 - precision_9: 0.9827 - recall_9: 0.9827 - val_loss: 0.0501 - val_accuracy: 0.9865 - val_auc_9: 0.9971 - val_precision_9: 0.9865 - val_recall_9: 0.9865
Epoch 42/150
epoch_end | time:  30.04.2021 11:40:19
 — val_aps:  0.492082 — val_a: 0.969062
1547/1547 - 767s - loss: 0.0097 - accuracy: 0.9831 - auc_9: 0.9981 - precision_9: 0.9831 - recall_9: 0.9831 - val_loss: 0.0449 - val_accuracy: 0.9876 - val_auc_9: 0.9976 - val_precision_9: 0.9876 - val_recall_9: 0.9876
Epoch 43/150
epoch_end | time:  30.04.2021 11:50:56
 — val_aps:  0.

...

model_cnn_china_grid_False_5_10_30_8_200501_0 valid_for_train:  0.9437851423015682 0.5237000563728019 | test:  0.9435337051226829 0.5098771144824633
128 (99001, 128) (False, 0.0001, 3, 10, 100, 0.25, 0.0002, 20, 6)
train_begin | time:  01.05.2021 00:07:11
Epoch 1/150
epoch_end | time:  01.05.2021 00:17:44
 — val_aps:  0.398812 — val_a: 0.960268
1547/1547 - 650s - loss: 0.0371 - accuracy: 0.9541 - auc_11: 0.9870 - precision_11: 0.9541 - recall_11: 0.9541 - val_loss: 0.1295 - val_accuracy: 0.9566 - val_auc_11: 0.9898 - val_precision_11: 0.9566 - val_recall_11: 0.9566
Epoch 2/150
epoch_end | time:  01.05.2021 00:28:28
 — val_aps:  0.441028 — val_a: 0.967167
1547/1547 - 645s - loss: 0.0212 - accuracy: 0.9721 - auc_11: 0.9926 - precision_11: 0.9721 - recall_11: 0.9721 - val_loss: 0.1065 - val_accuracy: 0.9721 - val_auc_11: 0.9904 - val_precision_11: 0.9721 - val_recall_11: 0.9721
Epoch 3/150
epoch_end | time:  01.05.2021 00:38:52
 — val_aps:  0.430057 — val_a: 0.962782
1547/1547 - 625s - lo

Epoch 26/150
epoch_end | time:  01.05.2021 04:42:30
 — val_aps:  0.467573 — val_a: 0.969781
1547/1547 - 645s - loss: 0.0120 - accuracy: 0.9810 - auc_11: 0.9972 - precision_11: 0.9810 - recall_11: 0.9810 - val_loss: 0.0800 - val_accuracy: 0.9696 - val_auc_11: 0.9952 - val_precision_11: 0.9696 - val_recall_11: 0.9696
Epoch 27/150
epoch_end | time:  01.05.2021 04:53:04
 — val_aps:  0.519548 — val_a: 0.971692
1547/1547 - 630s - loss: 0.0120 - accuracy: 0.9808 - auc_11: 0.9972 - precision_11: 0.9808 - recall_11: 0.9808 - val_loss: 0.0562 - val_accuracy: 0.9836 - val_auc_11: 0.9972 - val_precision_11: 0.9836 - val_recall_11: 0.9836
Epoch 28/150
epoch_end | time:  01.05.2021 05:03:20
 — val_aps:  0.484291 — val_a: 0.966544
1547/1547 - 614s - loss: 0.0118 - accuracy: 0.9813 - auc_11: 0.9973 - precision_11: 0.9813 - recall_11: 0.9813 - val_loss: 0.0482 - val_accuracy: 0.9870 - val_auc_11: 0.9971 - val_precision_11: 0.9870 - val_recall_11: 0.9870
Epoch 29/150
epoch_end | time:  01.05.2021 05:13:

Epoch 9/150
epoch_end | time:  01.05.2021 09:35:08
 — val_aps:  0.471066 — val_a: 0.967104
1547/1547 - 816s - loss: 0.0139 - accuracy: 0.9795 - auc_12: 0.9961 - precision_12: 0.9795 - recall_12: 0.9795 - val_loss: 0.0532 - val_accuracy: 0.9864 - val_auc_12: 0.9980 - val_precision_12: 0.9864 - val_recall_12: 0.9864
Epoch 10/150
epoch_end | time:  01.05.2021 09:48:46
 — val_aps:  0.423509 — val_a: 0.965208
1547/1547 - 815s - loss: 0.0136 - accuracy: 0.9798 - auc_12: 0.9963 - precision_12: 0.9798 - recall_12: 0.9798 - val_loss: 0.0833 - val_accuracy: 0.9695 - val_auc_12: 0.9956 - val_precision_12: 0.9695 - val_recall_12: 0.9695
Epoch 11/150
epoch_end | time:  01.05.2021 10:01:58
 — val_aps:  0.477458 — val_a: 0.971535
1547/1547 - 793s - loss: 0.0133 - accuracy: 0.9801 - auc_12: 0.9964 - precision_12: 0.9801 - recall_12: 0.9801 - val_loss: 0.0767 - val_accuracy: 0.9749 - val_auc_12: 0.9959 - val_precision_12: 0.9749 - val_recall_12: 0.9749
Epoch 12/150
epoch_end | time:  01.05.2021 10:15:2

Epoch 35/150
epoch_end | time:  01.05.2021 16:11:27
 — val_aps:  0.454228 — val_a: 0.966851
1547/1547 - 789s - loss: 0.0108 - accuracy: 0.9821 - auc_12: 0.9977 - precision_12: 0.9821 - recall_12: 0.9821 - val_loss: 0.0686 - val_accuracy: 0.9786 - val_auc_12: 0.9966 - val_precision_12: 0.9786 - val_recall_12: 0.9786
Epoch 36/150
epoch_end | time:  01.05.2021 16:24:50
 — val_aps:  0.469754 — val_a: 0.969417
1547/1547 - 801s - loss: 0.0106 - accuracy: 0.9823 - auc_12: 0.9978 - precision_12: 0.9823 - recall_12: 0.9823 - val_loss: 0.0598 - val_accuracy: 0.9808 - val_auc_12: 0.9970 - val_precision_12: 0.9808 - val_recall_12: 0.9808
Epoch 37/150
epoch_end | time:  01.05.2021 16:37:58
 — val_aps:  0.425705 — val_a: 0.970392
1547/1547 - 785s - loss: 0.0105 - accuracy: 0.9821 - auc_12: 0.9979 - precision_12: 0.9821 - recall_12: 0.9821 - val_loss: 0.0639 - val_accuracy: 0.9802 - val_auc_12: 0.9965 - val_precision_12: 0.9802 - val_recall_12: 0.9802
Epoch 38/150
epoch_end | time:  01.05.2021 16:51:

Epoch 15/150
epoch_end | time:  02.05.2021 00:05:36
 — val_aps:  0.475461 — val_a: 0.974052
1547/1547 - 1217s - loss: 0.0113 - accuracy: 0.9823 - auc_13: 0.9975 - precision_13: 0.9823 - recall_13: 0.9823 - val_loss: 0.0540 - val_accuracy: 0.9837 - val_auc_13: 0.9973 - val_precision_13: 0.9837 - val_recall_13: 0.9837
Epoch 16/150
epoch_end | time:  02.05.2021 00:25:39
 — val_aps:  0.456239 — val_a: 0.967380
1547/1547 - 1201s - loss: 0.0111 - accuracy: 0.9824 - auc_13: 0.9977 - precision_13: 0.9824 - recall_13: 0.9824 - val_loss: 0.0739 - val_accuracy: 0.9734 - val_auc_13: 0.9963 - val_precision_13: 0.9734 - val_recall_13: 0.9734
Epoch 17/150
epoch_end | time:  02.05.2021 00:45:59
 — val_aps:  0.449260 — val_a: 0.971095
1547/1547 - 1221s - loss: 0.0112 - accuracy: 0.9819 - auc_13: 0.9976 - precision_13: 0.9819 - recall_13: 0.9819 - val_loss: 0.2534 - val_accuracy: 0.9114 - val_auc_13: 0.9698 - val_precision_13: 0.9114 - val_recall_13: 0.9114
Epoch 18/150
epoch_end | time:  02.05.2021 01: